# DLT

*Direct Linear Transform*

Método sencillo para estimar transformaciones proyectivas mediante la solución de un sistema de ecuaciones lineal.

In [ ]:
%matplotlib inline

import numpy             as np
import cv2               as cv
import numpy.linalg      as la
import matplotlib.pyplot as plt

from matplotlib.pyplot   import imshow, subplot, plot

from umucv.htrans import htrans, homog, kgen, null1

def fig(w,h):
    plt.figure(figsize=(w,h))

def readrgb(file):
    return cv.cvtColor( cv.imread('../images/'+file), cv.COLOR_BGR2RGB) 

def rgb2gray(x):
    return cv.cvtColor(x,cv.COLOR_RGB2GRAY)

## matriz fundamental

Algoritmo de los 8 puntos (o más).

Cada correspondencia entre puntos en la imagen izquierda $(x,y,1)$ y derecha $(p,q,1)$, datos conocidos, da lugar a una ecuación sobre los elementos de la matriz F.

$$\begin{bmatrix} x & y& 1\end{bmatrix} \begin{bmatrix} f_1 & f_2 & f_3 \\ f_4 & f_5 & f_6  \\ f_7 & f_8 & f_9 \end{bmatrix} \begin{bmatrix} p \\ q \\1\end{bmatrix} = 0$$

Esto es equivalente a

$$ \begin{bmatrix} px & py &  p & qx & qy & q & x & y & 1\end{bmatrix} \begin{bmatrix} f_1 \\ f_2 \\f_3 \\ f_4 \\ f_5 \\ f_6  \\ f_7 \\ f_8 \\ f_9 \end{bmatrix} = 0$$

Cada fila de la matriz de coeficientes (ecuación homogénea generada por cada correspondencia en el sistema cuya solución es F) se puede obtener como el "outer product" de los puntos correspondientes.

In [ ]:
v1 = np.array(
      [[ 278.,  343.],
       [ 335.,  312.],
       [ 386.,  279.],
       [ 433.,  254.],
       [ 270.,  304.],
       [ 332.,  272.],
       [ 389.,  238.],
       [ 434.,  210.],
       [ 260.,  253.],
       [ 324.,  218.],
       [ 389.,  186.],
       [ 442.,  160.],
       [ 244.,  189.],
       [ 317.,  158.],
       [ 390.,  125.],
       [ 447.,   97.],
       [ 204.,  147.],
       [ 276.,  114.],
       [ 340.,   87.],
       [ 395.,   64.],
       [ 172.,  107.],
       [ 238.,   83.],
       [ 302.,   55.],
       [ 355.,   39.],
       [ 146.,   81.],
       [ 210.,   58.],
       [ 273.,   37.],
       [ 318.,   21.]])

v2 = np.array(
      [[ 184.,  399.],
       [ 226.,  418.],
       [ 277.,  434.],
       [ 338.,  460.],
       [ 171.,  358.],
       [ 216.,  374.],
       [ 273.,  394.],
       [ 331.,  412.],
       [ 155.,  307.],
       [ 203.,  321.],
       [ 262.,  342.],
       [ 324.,  358.],
       [ 138.,  251.],
       [ 191.,  265.],
       [ 252.,  284.],
       [ 321.,  299.],
       [ 186.,  218.],
       [ 233.,  227.],
       [ 291.,  240.],
       [ 357.,  255.],
       [ 226.,  188.],
       [ 269.,  196.],
       [ 327.,  206.],
       [ 387.,  220.],
       [ 257.,  170.],
       [ 299.,  177.],
       [ 352.,  185.],
       [ 409.,  194.]])

x1 = rgb2gray(readrgb('cube3.png'))
x2 = rgb2gray(readrgb('cube4.png'))

fig(12,4)
subplot(1,2,1)
imshow(x1,'gray'); ax = plt.axis()
plot(v1[:,0],v1[:,1],'r.'); plt.axis(ax)
subplot(1,2,2)
imshow(x2,'gray')
plot(v2[:,0],v2[:,1],'r.'); plt.axis(ax);

Cuando no hay outliers la matriz $F$ se puede obtener resolviendo un sencillo sistema de ecuaciones. En la práctica es mejor usar la siguiente función de OpenCV que admite correspondencias incorrectas.

In [ ]:
F,_ = cv.findFundamentalMat(v1,v2,cv.FM_LMEDS)
F

Comprobemos que funciona:

In [ ]:
[ x2 @ F @ x1 for x1,x2 in zip(homog(v1),homog(v2)) ]

In [ ]:
sum([ abs(x2 @ F @ x1) for x1,x2 in zip(homog(v1),homog(v2)) ])

No son ceros perfectos, pero el residuo es razonablemente pequeño, puede no estar mal del todo.

Vamos a calcular nosotros mismos la matriz fundamental resolviendo un sistema de ecuaciones homogéneo. Para que el sistema esté bien condicionado numéricamente es necesario trabajar con coordenadas del orden de 1, en vez de coordenadas de pixels, que al estar multiplicadas entre sí producen elementos de magnitudes muy descompensadas. Por tanto, vamos a normalizar los pixels, quitando la transformación de calibración, de modo que obtendremos inicialmente una aproximación a la matriz Esencial.

In [ ]:
K = kgen((640,480),1.6)

hn1 = homog(v1) @ la.inv(K).T
hn2 = homog(v2) @ la.inv(K).T

Formamos la matriz de coefientes del sistema homogéneo y lo resolvemos con `null1` (en su momento vimos (en el notebook de [sistemas de ecuaciones](sistecs.ipynb)) que es un sencillo algoritmo basado en la descomposición en valores singulares).

In [ ]:
dat = np.array([np.outer(x,y) for x,y in zip(hn2,hn1) ])
myE = null1(dat.reshape(-1,9)).reshape(3,3)

In [ ]:
myE

In [ ]:
[ x2 @ myE @ x1 for x1,x2 in zip(hn1,hn2) ]

Se cumple bastante bien la restricción en todas las correspondencias. Es muy simple ahora conseguir la matriz fundamental, que opera directamente con coordenadas de pixel:

In [ ]:
myF = la.inv(K).T @ myE @ la.inv(K)

In [ ]:
[ x2 @ myF @ x1 for x1,x2 in zip(homog(v1),homog(v2)) ]

Las condiciones se cumplen también muy bien sobre pixels crudos. Y los valores numéricos de nuestra matriz Fundamental son muy parecidos a los conseguidos por OpenCV. (Para comparar las matrices, que son homogéneas, es necesario ponerlas con una escala común, p.ej., dividiendo todo por el mayor elemento):

In [ ]:
myF = myF/myF[2,2] 
myF

In [ ]:
F

Incluso conseguimos un residuo menor (!?):

In [ ]:
sum([ abs(x2 @ myF @ x1) for x1,x2 in zip(homog(v1),homog(v2)) ])

Esto no significa que este método tan simple sea mejor que el de OpenCV. Lo que ocurre es que no hemos impuesto a nuestra $F$ una condición importante: que todas las líneas epipolares generadas pasen por un punto común: el "epipolo", que es la imagen del centro de la otra cámara. Matemáticamente esto significa que $F$ tiene que tener rango 2, o sea, su tercer valor singular nulo.

In [ ]:
la.svd(F)[1]

In [ ]:
la.svd(myF)[1]

En nuestro caso $s_3$ es $\sim 100$ veces menor que el $s_2$, que no está mal, pero no es un cero numérico.

## homografía

Dado un conjunto de correspondencias  $(x,y,1) \leftrightarrow (p,q,1)$ relacionadas por una homografía, cada una da lugar a una ecuación

$$\begin{bmatrix} x \\ y \\1\end{bmatrix} = \lambda \begin{bmatrix} h_1 & h_2 & h_3 \\ h_4 & h_5 & h_6  \\ h_7 & h_8 & h_9 \end{bmatrix}  \begin{bmatrix} p \\ q \\1\end{bmatrix}$$

Se desconoce la escala homogénea $\lambda$ de cada ecuación. Pero es posible transformarlo a 3 ecuaciones homogéneas. Como ambos lados de la ecuación son vectores proporcionales, su producto vectorial (cross) debe ser cero.

$$ \begin{bmatrix} x \\ y \\1\end{bmatrix} \times \begin{bmatrix} h_1 & h_2 & h_3 \\ h_4 & h_5 & h_6  \\ h_7 & h_8 & h_9 \end{bmatrix}  \begin{bmatrix} p \\ q \\1\end{bmatrix} = \begin{bmatrix} 0 \\ 0 \\0\end{bmatrix} $$

Desarrollando esto podemos construir 3 ecuaciones homogéneas (2 de ellas independientes) para los elementos de la homografía.

$$\begin{bmatrix} 0 & 0 & 0 & p & q & 1 & -p y & -q y & -y \\ 
            -p & -q & -1& 0& 0& 0& p x& q x& x\\
            p y& q y& y& -p x& -q x& -x& 0& 0 & 0 \end{bmatrix} \begin{bmatrix} h_1 \\ h_2 \\h_3 \\ h_4 \\ h_5 \\ h_6  \\ h_7 \\ h_8 \\ h_9 \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \\0\end{bmatrix} $$



In [ ]:
X = np.array(
   [[0,   0  ],
    [0,   1  ],
    [0.5, 1  ],
    [0.5, 0.5],
    [1,   0.5],
    [1,   0  ]])

Y = np.array(
   [[ 260.,  320.],
    [ 192.,  272.],
    [ 267.,  260.],
    [ 304.,  278.],
    [ 374.,  266.],
    [ 425.,  285.]])

Estos datos de prueba los hemos creado con una transformación conocida.

In [ ]:
H = np.array([[250,-11, 260],
              [22,  33, 320],
              [0.2,  0.3, 1]])

np.round(htrans(H,X))

In [ ]:
def eqs(x,y):
    x_0, x_1 = x
    y_0, y_1 = y
    return [[0, 0, 0, x_0, x_1, 1, -x_0*y_1, -x_1*y_1, -y_1], 
            [-x_0, -x_1, -1, 0, 0, 0, x_0*y_0, x_1*y_0, y_0],
            [x_0*y_1, x_1*y_1, y_1, -x_0*y_0, -x_1*y_0, -y_0, 0, 0, 0]]

In [ ]:
eqs(X[0],Y[0])

In [ ]:
A = sum([eqs(x,y) for x,y in zip(X,Y)],[])

In [ ]:
myH = null1(A).reshape(3,3)
myH

In [ ]:
myH = myH/myH[2,2]
myH

In [ ]:
htrans(myH,X)

In [ ]:
Y

## camera resection

pendiente

## triangulation

pendiente